In [1]:
!pip3 install twilio

In [2]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [11]:
query = 'Costa Rica'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=fc4fa8d454a647f6af9222735232310&q=Costa Rica&days=1&aqi=no&alerts=no'

In [12]:
response = requests.get(url_clima).json()

In [13]:
response

{'location': {'name': 'San Jose',
  'region': 'San Jose',
  'country': 'Costa Rica',
  'lat': 9.93,
  'lon': -84.08,
  'tz_id': 'America/Costa_Rica',
  'localtime_epoch': 1698165250,
  'localtime': '2023-10-24 10:34'},
 'current': {'last_updated_epoch': 1698165000,
  'last_updated': '2023-10-24 10:30',
  'temp_c': 25.0,
  'temp_f': 77.0,
  'is_day': 1,
  'condition': {'text': 'Overcast',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png',
   'code': 1009},
  'wind_mph': 5.6,
  'wind_kph': 9.0,
  'wind_degree': 110,
  'wind_dir': 'ESE',
  'pressure_mb': 1018.0,
  'pressure_in': 30.06,
  'precip_mm': 1.2,
  'precip_in': 0.05,
  'humidity': 65,
  'cloud': 100,
  'feelslike_c': 27.4,
  'feelslike_f': 81.4,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 5.0,
  'gust_mph': 3.4,
  'gust_kph': 5.4},
 'forecast': {'forecastday': [{'date': '2023-10-24',
    'date_epoch': 1698105600,
    'day': {'maxtemp_c': 21.7,
     'maxtemp_f': 71.0,
     'mintemp_c': 15.7,
     'mintemp_f': 60.2,
   

In [27]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [28]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [16]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [17]:
response['forecast']['forecastday'][0]['hour'][0]

{'time_epoch': 1698127200,
 'time': '2023-10-24 00:00',
 'temp_c': 16.6,
 'temp_f': 61.8,
 'is_day': 0,
 'condition': {'text': 'Fog',
  'icon': '//cdn.weatherapi.com/weather/64x64/night/248.png',
  'code': 1135},
 'wind_mph': 1.6,
 'wind_kph': 2.5,
 'wind_degree': 227,
 'wind_dir': 'SW',
 'pressure_mb': 1012.0,
 'pressure_in': 29.89,
 'precip_mm': 0.04,
 'precip_in': 0.0,
 'humidity': 99,
 'cloud': 100,
 'feelslike_c': 16.6,
 'feelslike_f': 61.8,
 'windchill_c': 16.6,
 'windchill_f': 61.8,
 'heatindex_c': 16.6,
 'heatindex_f': 61.8,
 'dewpoint_c': 16.5,
 'dewpoint_f': 61.7,
 'will_it_rain': 0,
 'chance_of_rain': 69,
 'will_it_snow': 0,
 'chance_of_snow': 0,
 'vis_km': 0.0,
 'vis_miles': 0.0,
 'gust_mph': 2.9,
 'gust_kph': 4.7,
 'uv': 1.0}

In [29]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2023-10-24'

In [34]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [35]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [36]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

57

# Dataframe

In [39]:
def get_forecast(response,i):
    
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = float(response['forecast']['forecastday'][0]['hour'][i]['temp_c'])
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,tempe,rain,prob_rain

In [40]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████| 24/24 [00:00<?, ?it/s]


In [41]:
datos[0]

('2023-10-24', 0, 'Fog', 16.6, 0, 69)

In [42]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-10-24,0,Fog,16.6,0,69
1,2023-10-24,1,Light rain shower,16.6,0,66
2,2023-10-24,2,Light rain shower,16.6,0,57
3,2023-10-24,3,Light rain shower,16.2,1,89
4,2023-10-24,4,Fog,16.2,0,0
5,2023-10-24,5,Mist,16.1,0,0
6,2023-10-24,6,Mist,18.1,0,63
7,2023-10-24,7,Partly cloudy,20.2,0,0
8,2023-10-24,8,Patchy rain possible,21.7,1,78
9,2023-10-24,9,Light rain shower,23.0,1,97


In [ ]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain
    
# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [43]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
3,2023-10-24,3,Light rain shower,16.2,1,89
8,2023-10-24,8,Patchy rain possible,21.7,1,78
9,2023-10-24,9,Light rain shower,23.0,1,97
10,2023-10-24,10,Overcast,25.0,1,97
11,2023-10-24,11,Light rain shower,22.8,1,96
12,2023-10-24,12,Light rain shower,23.1,1,96
13,2023-10-24,13,Light rain shower,22.7,1,96
14,2023-10-24,14,Light rain shower,21.7,1,95
15,2023-10-24,15,Light rain shower,20.8,1,95
16,2023-10-24,16,Moderate or heavy rain shower,19.3,1,95


In [44]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [45]:
df['Fecha'][0]

'2023-10-24'

In [47]:
df_rain

,Condicion
Hora,
8,Patchy rain possible
9,Light rain shower
10,Overcast
11,Light rain shower
12,Light rain shower
13,Light rain shower
14,Light rain shower
15,Light rain shower
16,Moderate or heavy rain shower


In [48]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2023-10-24 en Costa Rica es : \n\n\n                           Condicion\nHora                               \n8              Patchy rain possible\n9                 Light rain shower\n10                         Overcast\n11                Light rain shower\n12                Light rain shower\n13                Light rain shower\n14                Light rain shower\n15                Light rain shower\n16    Moderate or heavy rain shower\n17                Light rain shower\n18    Moderate or heavy rain shower\n19                    Moderate rain\n20    Moderate or heavy rain shower\n21                Light rain shower'

In [31]:
PHONE_NUMBER

'+1 614 401 3161'

# Mensaje SMS desde Twilio

In [50]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+50683659591'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SM866f0101579197712cfe77a3c3a81977
